In [1]:
import cv2
import numpy as np
import pyautogui
from ultralytics import YOLO

C:\Users\Khaled\AppData\Roaming\Python\Python312\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
model_path = 'Yolos/Varroa/3rd dataset/train38/weights/best.pt'  
model = YOLO(model_path)

In [3]:
threshold = 0.1

In [4]:
screen_width, screen_height = pyautogui.size()

region = (0, 0, screen_width, screen_height)

window_name = "YOLO Detection on Screen"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)  
cv2.resizeWindow(window_name, 800, 600)  

while True:
    screenshot = pyautogui.screenshot(region=region)
    frame = np.array(screenshot)

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    results = model.predict(source=frame, conf=threshold, verbose=False)

    detections = results[0].boxes.data.cpu().numpy()  
    class_counts = {}
    for detection in detections:
        class_id = int(detection[5])  
        class_counts[class_id] = class_counts.get(class_id, 0) + 1
        
    annotated_frame = results[0].plot()
    
    y_offset = 20
    for class_id, count in class_counts.items():
        class_name = model.names[class_id]  
        text = f"{class_name}: {count}"
        cv2.putText(
            annotated_frame,
            text,
            (10, y_offset),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2,
            cv2.LINE_AA,
        )
        y_offset += 30
    cv2.imshow(window_name, annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()